In [1]:
using CSV, DataFrames, DelimitedFiles, JuMP, Gurobi, LinearAlgebra

In [371]:
demands = CSV.read("data/processed/demands.csv", DataFrame)
fire_dists =  readdlm("data/processed/fire_distances.csv", ',')
base_fire_dists =  readdlm("data/processed/base_fire_distances.csv", ',');

f = maximum(demands[!, "fire_id"])
tau = convert(Array{Int}, ones((f, f)))
tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))));

In [372]:
demands = demands[18:92, :];

In [ ]:
demands = demands[1:100, :];

In [373]:
c = size(base_fire_dists)[1]
min_f = minimum(demands[!, "fire_id"])
f = maximum(demands[!, "fire_id"])
d = maximum(demands[!, "day"])
min_d = minimum(demands[!, "day"])
n = size(demands)[1]


break_length = 2
max_consecutive = 14

14

In [225]:
f

25

In [367]:
rest_by = [44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 47, 48, 48, 49, 
           50, 50, 50, 50, 51, 52, 52, 53, 54, 55, 56, 56, 57, 58, 59]

start_at = [26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 
           26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41];

In [476]:
non_rested = 
[(i, demands[k, "fire_id"], 
        demands[l, "fire_id"], 
        demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
        demands[l, "day"], 0) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"]) &
         (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] <= rest_by[i])
     ]
rested = 
[(i, demands[k, "fire_id"], 
       demands[l, "fire_id"], 
       demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
       demands[l, "day"],
       1
      ) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"])
     ]
from_start = [(i, 
               start_at[i], 
               demands[k, "fire_id"], 
               min_d - 1, 
               min_d - 1 + tau[demands[k, "fire_id"], start_at[i]], 
               0) 
    for i=1:c, k=1:n 
    if demands[k, "day"] == min_d - 1 + tau[demands[k, "fire_id"], start_at[i]]
    ]
to_start = [(i, 
             start_at[i], 
             start_at[i], 
             min_d - 2, 
             min_d - 1,
             0) 
    for i=1:c
    ]

30-element Vector{NTuple{6, Int64}}:
 (1, 26, 26, 42, 43, 0)
 (2, 27, 27, 42, 43, 0)
 (3, 28, 28, 42, 43, 0)
 (4, 29, 29, 42, 43, 0)
 (5, 30, 30, 42, 43, 0)
 (6, 31, 31, 42, 43, 0)
 (7, 32, 32, 42, 43, 0)
 (8, 34, 34, 42, 43, 0)
 (9, 35, 35, 42, 43, 0)
 (10, 36, 36, 42, 43, 0)
 (11, 37, 37, 42, 43, 0)
 (12, 38, 38, 42, 43, 0)
 (13, 39, 39, 42, 43, 0)
 ⋮
 (19, 29, 29, 42, 43, 0)
 (20, 30, 30, 42, 43, 0)
 (21, 31, 31, 42, 43, 0)
 (22, 32, 32, 42, 43, 0)
 (23, 34, 34, 42, 43, 0)
 (24, 35, 35, 42, 43, 0)
 (25, 36, 36, 42, 43, 0)
 (26, 37, 37, 42, 43, 0)
 (27, 38, 38, 42, 43, 0)
 (28, 39, 39, 42, 43, 0)
 (29, 40, 40, 42, 43, 0)
 (30, 41, 41, 42, 43, 0)

30-element Vector{Int64}:
 26
 27
 28
 29
 30
 31
 32
 34
 35
 36
 37
 38
 39
  ⋮
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23
 24
 25

15-element Vector{Int64}:
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23
 24
 25

In [588]:
m4 = Model(Gurobi.Optimizer)

# make fire to fire travel index set
# (crew, fire_from, fire_to, time_leave, time_arrive, rested)
non_rested = 
[(i, demands[k, "fire_id"], 
        demands[l, "fire_id"], 
        demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
        demands[l, "day"], 0) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"]) &
         (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] <= rest_by[i])
     ]
rested = 
[(i, demands[k, "fire_id"], 
       demands[l, "fire_id"], 
       demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
       demands[l, "day"],
       1
      ) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"])
     ]
from_start = [(i, 
               start_at[i], 
               demands[k, "fire_id"], 
               min_d - 1, 
               min_d - 1 + tau[demands[k, "fire_id"], start_at[i]], 
               0) 
    for i=1:c, k=1:n 
    if demands[k, "day"] == min_d - 1 + tau[demands[k, "fire_id"], start_at[i]]
    ]
to_start = [(i, 
             start_at[i], 
             start_at[i], 
             min_d - 2, 
             min_d - 1,
             0) 
    for i=1:c
    ]

S = vcat(non_rested, rested)
S = vcat(from_start, S)
S = vcat(to_start, S)
@variable(m4, fire_to_fire[S] >= 0, Int)

# make fire to rest travel index set
# (crew, fire, time_leave, time_arrive, rested)
non_rested = [(i, 
               demands[k, "fire_id"], 
               demands[k, "day"], 
               demands[k, "day"] + tau_base_to_fire[i, demands[k, "fire_id"]], 
               0)
    for i=1:c, k=1:n if demands[k, "day"] < rest_by[i]]
rested = [(i, 
           demands[k, "fire_id"], 
           demands[k, "day"], 
           demands[k, "day"] + tau_base_to_fire[i, demands[k, "fire_id"]], 
           1)
    for i=1:c, k=1:n if i>0]

from_start = [(i, 
               start_at[i], 
               min_d - 1, 
               min_d - 1 + tau_base_to_fire[i, start_at[i]], 
               0) 
             for i=1:c]

S = vcat(non_rested, rested)
S = vcat(from_start, S)
@variable(m4, fire_to_rest[S] >= 0, Int)

# make rest to fire travel index set
# (crew, fire, time_leave, time_arrive, rested_start, rested_end)
to_rested = [(i, 
              demands[k, "fire_id"], 
              demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] - break_length,
              demands[k, "day"],
              rested,
              1) 
      for i=1:c, k=1:n, rested=0:1
      if demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] - break_length >= min_d
     ]
to_not_rested = [(i, 
              demands[k, "fire_id"], 
              demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]],
              demands[k, "day"],
              0,
              0) 
      for i=1:c, k=1:n
      if demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] >= min_d
     ]

S = vcat(to_rested, to_not_rested)
T = [s for s in S if (s[3] <= rest_by[s[1]]) | s[5] == 1]
@variable(m4, rest_to_fire[T] >= 0, Int)

# make rest to rest index set
# (crew, time_leave, time_arrive, rested_start, rested_end)
to_rested = [(i, t, t + break_length, rest, 1) for i=1:c, t=min_d-1:d, rest=0:1 if c > 0]
to_not_rested = [(i, t, t + 1, rest, rest) for i=1:c, t=min_d-1:d, rest=0:1 if c > 0]
S = vcat(to_rested, to_not_rested)
T = [s for s in S if (s[2] <= rest_by[s[1]]) | s[4] == 1]
@variable(m4, rest_to_rest[T] >= 0)

# flow constraints
f_to_f_keys_arr = [[s[1] for s in keys(fire_to_fire) if s[1][1] == i] for i=1:c]
r_to_f_keys_arr = [[s[1] for s in keys(rest_to_fire) if s[1][1] == i] for i=1:c]
f_to_r_keys_arr = [[s[1] for s in keys(fire_to_rest) if s[1][1] == i] for i=1:c]

# TODO TODO branch off the starting constraints because so many 0=0 slows build
fires = unique(vcat(unique(start_at), Vector(min_f:f)))
@constraint(m4, fire_flow[i=1:c, f=fires, t=min_d-1:d-1, rest=0:1], 
    
            # outflow
            sum(fire_to_fire[key] for key in f_to_f_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[4] == t) & (key[6] == rest)
                ) + 
    
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[3] == t) & (key[5] == rest)
                ) 
    ==
            # inflow
            sum(fire_to_fire[key] for key in f_to_f_keys_arr[i]
                        if (key[1] == i) & (key[3] == f) & (key[5] == t) & (key[6] == rest)
                ) +
    
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[4] == t) & (key[6] == rest)
                ) 
    
            )

r_to_r_keys = [key[1] for key in keys(rest_to_rest)]
@constraint(m4, rest_flow[i=1:c, t=min_d-1:d-1, rest=0:1], 
    
            # outflow
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i]
                        if (key[1] == i) &  (key[3] == t) & (key[5] == rest)
                ) +
    
            sum(rest_to_rest[key] for key in r_to_r_keys
                        if (key[1] == i) &  (key[2] == t) & (key[4] == rest)
                )
    
            ==       
    
            # inflow
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i]
                        if (key[1] == i) &  (key[4] == t) &  (key[5] == rest)
                ) +
            sum(rest_to_rest[key] for key in r_to_r_keys
                        if (key[1] == i) &  (key[3] == t) & (key[5] == rest)
                )
           )

f_to_f_keys = [key[1] for key in keys(fire_to_fire)]
r_to_f_keys = [key[1] for key in keys(rest_to_fire)]
@constraint(m4, dem[row=1:n], 
            sum(fire_to_fire[key] for key in f_to_f_keys
                if (key[3] == demands[row, "fire_id"]) & 
                   (key[5] == demands[row, "day"])
                ) +
            sum(rest_to_fire[key] for key in r_to_f_keys
                if (key[2] == demands[row, "fire_id"]) & 
                   (key[4] == demands[row, "day"])
                )
    >= demands[row, "demand"])

start_f_to_f_keys = [key[1] for key in keys(fire_to_fire) if key[1][4] < min_d-1]

@constraint(m4, start, fire_to_fire[start_f_to_f_keys] .== 1)

start_r_to_r_keys = [key[1] for key in keys(rest_to_rest) if key[1][2] < min_d]
@constraint(m4, start_rest, rest_to_rest[start_r_to_r_keys] .== 0)

f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
f_to_r_keys = [s[1] for s in keys(fire_to_rest)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]
@objective(m4, Min, sum(fire_to_fire[key] * fire_dists[key[2], key[3]] for key in f_to_f_keys) +
                   sum(fire_to_rest[key] * base_fire_dists[key[1], key[2]] for key in f_to_r_keys) + 
                   sum(rest_to_fire[key] * base_fire_dists[key[1], key[2]] for key in r_to_f_keys)
);

Academic license - for non-commercial use only


In [589]:
m4

A JuMP Model
Minimization problem with:
Variables: 28424
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 26190 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 75 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 28424 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 27056 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: dem, fire_flow, fire_to_fire, fire_to_rest, rest_flow, rest_to_fire, rest_to_rest, start, start_rest

In [590]:
f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
f_to_r_keys = [s[1] for s in keys(fire_to_rest)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]
@objective(m4, Min, sum(fire_to_fire[key] * (1000 + fire_dists[key[2], key[3]]) for key in f_to_f_keys) +
                   sum(fire_to_rest[key] * (base_fire_dists[key[1], key[2]]) for key in f_to_r_keys) + 
                   sum(rest_to_fire[key] * (1000 + base_fire_dists[key[1], key[2]]) for key in r_to_f_keys)
);

In [591]:
optimize!(m4)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 26265 rows, 28424 columns and 79547 nonzeros
Model fingerprint: 0x3274c578
Variable types: 1368 continuous, 27056 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 23087 rows and 3881 columns
Presolve time: 2.07s
Presolved: 3178 rows, 24543 columns, 69323 nonzeros
Variable types: 761 continuous, 23782 integer (2357 binary)

Root relaxation: objective 2.713758e+05, 9766 iterations, 1.31 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 271375.840    0  214          - 271375.840      -     -    3s
H    0     0                    274253.73642 271375.840  1.05%     -    4s
H    0     0                    271375.87458 271375.840  0.00%     -    5s

Ex

In [592]:
unique(value.(fire_to_fire))

81-element Vector{Float64}:
  1.0
 -0.0
  0.0
  3.322222327049113e-7
  0.9999996668343639
  0.9999996668549354
  0.9999993335719783
  3.326812154069602e-7
  6.654166876320449e-7
  0.9999993335757353
  0.9999996669129723
  0.9999993335796284
  6.651082708089195e-7
  ⋮
  0.9999996668949528
  0.9999996668981526
  3.3270649228784054e-7
  0.999999333509352
  3.3308805923265483e-7
  0.9999996668635813
  6.664877253149866e-7
  0.9999996668844984
  3.3133334381602243e-7
  3.31904772387451e-7
  0.9999996669124172
  3.331620503470993e-7

In [580]:
rest_flow[10, 44, 0]

rest_flow[10,44,0] : -fire_to_rest[(10, 36, 43, 44, 0)] + rest_to_fire[(10, 13, 44, 47, 0, 1)] + rest_to_fire[(10, 14, 44, 47, 0, 1)] + rest_to_fire[(10, 15, 44, 47, 0, 1)] + rest_to_fire[(10, 16, 44, 47, 0, 1)] + rest_to_fire[(10, 17, 44, 47, 0, 1)] + rest_to_fire[(10, 18, 44, 47, 0, 1)] + rest_to_fire[(10, 19, 44, 47, 0, 1)] + rest_to_fire[(10, 12, 44, 47, 0, 1)] + rest_to_fire[(10, 12, 44, 45, 0, 0)] + rest_to_rest[(10, 44, 46, 0, 1)] - rest_to_rest[(10, 43, 44, 0, 0)] + rest_to_rest[(10, 44, 45, 0, 0)] == 0.0

In [533]:
fire_flow[10, 36, 43, 0]

LoadError: KeyError: key 36 not found

In [532]:
value(fire_to_rest[(10, 36, 43, 44, 0)])

17.0

In [531]:
value(rest_to_rest[(10, 44, 46, 0, 1)])

7.0

In [526]:
value.(rest_to_rest[[key[1] for key in keys(rest_to_rest) if (key[1][1] == 10) & (key[1][2] ==44)]])

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [(10, 44, 46, 0, 1), (10, 44, 46, 1, 1), (10, 44, 45, 0, 0), (10, 44, 45, 1, 1)]
And data, a 4-element Vector{Float64}:
 7.0
 0.0
 0.0
 0.0

In [574]:
m3 = Model(Gurobi.Optimizer)

# make fire to fire travel index set
# (crew, fire_from, fire_to, time_leave, time_arrive, rested)
non_rested = 
[(i, demands[k, "fire_id"], 
        demands[l, "fire_id"], 
        demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
        demands[l, "day"], 0) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"]) &
         (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] <= rest_by[i])
     ]
rested = 
[(i, demands[k, "fire_id"], 
       demands[l, "fire_id"], 
       demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
       demands[l, "day"],
       1
      ) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"])
     ]
from_start = [(i, 
               start_at[i], 
               demands[k, "fire_id"], 
               min_d - 1, 
               min_d - 1 + tau[demands[k, "fire_id"], start_at[i]], 
               0) 
    for i=1:c, k=1:n 
    if demands[k, "day"] == min_d - 1 + tau[demands[k, "fire_id"], start_at[i]]
    ]


S = vcat(non_rested, rested)
S = vcat(from_start, S)
@variable(m3, fire_to_fire[S] >= 0)

# make fire to rest travel index set
# (crew, fire, time_leave, time_arrive, rested)
non_rested = [(i, 
               demands[k, "fire_id"], 
               demands[k, "day"], 
               demands[k, "day"] + tau_base_to_fire[i, demands[k, "fire_id"]], 
               0)
    for i=1:c, k=1:n if demands[k, "day"] < rest_by[i]]
rested = [(i, 
           demands[k, "fire_id"], 
           demands[k, "day"], 
           demands[k, "day"] + tau_base_to_fire[i, demands[k, "fire_id"]], 
           1)
    for i=1:c, k=1:n if i>0]

from_start = [(i, 
               start_at[i], 
               min_d - 1, 
               min_d - 1 + tau_base_to_fire[i, start_at[i]], 
               0) 
             for i=1:c]

S = vcat(non_rested, rested)
S = vcat(from_start, S)
@variable(m3, fire_to_rest[S] >= 0)

# make rest to fire travel index set
# (crew, fire, time_leave, time_arrive, rested_start, rested_end)
to_rested = [(i, 
              demands[k, "fire_id"], 
              demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] - break_length,
              demands[k, "day"],
              rested,
              1) 
      for i=1:c, k=1:n, rested=0:1
      if demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] - break_length >= min_d
     ]
to_not_rested = [(i, 
              demands[k, "fire_id"], 
              demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]],
              demands[k, "day"],
              0,
              0) 
      for i=1:c, k=1:n
      if demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] >= min_d
     ]

S = vcat(to_rested, to_not_rested)
T = [s for s in S if (s[3] <= rest_by[s[1]]) | s[5] == 1]
@variable(m3, rest_to_fire[T] >= 0)

# make rest to rest index set
# (crew, time_leave, time_arrive, rested_start, rested_end)
to_rested = [(i, t, t + break_length, rest, 1) for i=1:c, t=min_d-1:d, rest=0:1 if c > 0]
to_not_rested = [(i, t, t + 1, rest, rest) for i=1:c, t=min_d-1:d, rest=0:1 if c > 0]
S = vcat(to_rested, to_not_rested)
T = [s for s in S if (s[2] <= rest_by[s[1]]) | s[4] == 1]
@variable(m3, rest_to_rest[T] >= 0)

# flow constraints
f_to_f_keys_arr = [[s[1] for s in keys(fire_to_fire) if s[1][1] == i] for i=1:c]
r_to_f_keys_arr = [[s[1] for s in keys(rest_to_fire) if s[1][1] == i] for i=1:c]
f_to_r_keys_arr = [[s[1] for s in keys(fire_to_rest) if s[1][1] == i] for i=1:c]

@constraint(m3, fire_flow[i=1:c, f=min_f:f, t=min_d:d-1, rest=0:1], 
    
            # outflow
            sum(fire_to_fire[key] for key in f_to_f_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[4] == t) & (key[6] == rest)
                ) + 
    
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[3] == t) & (key[5] == rest)
                ) 
    ==
            # inflow
            sum(fire_to_fire[key] for key in f_to_f_keys_arr[i]
                        if (key[1] == i) & (key[3] == f) & (key[5] == t) & (key[6] == rest)
                ) +
    
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[4] == t) & (key[6] == rest)
                ) 
    
            )

r_to_r_keys = [key[1] for key in keys(rest_to_rest)]
@constraint(m3, rest_flow[i=1:c, t=min_d:d-1, rest=0:1], 
    
            # outflow
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i]
                        if (key[1] == i) &  (key[3] == t) & (key[5] == rest)
                ) +
    
            sum(rest_to_rest[key] for key in r_to_r_keys
                        if (key[1] == i) &  (key[2] == t) & (key[4] == rest)
                )
    
            ==       
    
            # inflow
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i]
                        if (key[1] == i) &  (key[4] == t) &  (key[5] == rest)
                ) +
            sum(rest_to_rest[key] for key in r_to_r_keys
                        if (key[1] == i) &  (key[3] == t) & (key[5] == rest)
                )
           )

f_to_f_keys = [key[1] for key in keys(fire_to_fire)]
r_to_f_keys = [key[1] for key in keys(rest_to_fire)]
@constraint(m3, dem[row=1:n], 
            sum(fire_to_fire[key] for key in f_to_f_keys
                if (key[3] == demands[row, "fire_id"]) & 
                   (key[5] == demands[row, "day"])
                ) +
            sum(rest_to_fire[key] for key in r_to_f_keys
                if (key[2] == demands[row, "fire_id"]) & 
                   (key[4] == demands[row, "day"])
                )
    >= demands[row, "demand"])

start_f_to_f_keys = [key[1] for key in keys(fire_to_fire) if key[1][4] < min_d]
start_f_to_r_keys = [key[1] for key in keys(fire_to_rest) if key[1][3] < min_d]

@constraint(m3, start[i=1:c],
            sum(fire_to_fire[key] for key in start_f_to_f_keys if key[1] == i) + 
            sum(fire_to_rest[key] for key in start_f_to_r_keys if key[1] == i) == 1
           )

start_r_to_r_keys = [key[1] for key in keys(rest_to_rest) if key[1][2] < min_d]
@constraint(m3, start_rest, rest_to_rest[start_r_to_r_keys] .== 0)

f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
f_to_r_keys = [s[1] for s in keys(fire_to_rest)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]
@objective(m3, Min, sum(fire_to_fire[key] * (10000 + fire_dists[key[2], key[3]]) for key in f_to_f_keys) +
                   sum(fire_to_rest[key] * (base_fire_dists[key[1], key[2]]) for key in f_to_r_keys) + 
                   sum(rest_to_fire[key] * (10000 + base_fire_dists[key[1], key[2]]) for key in r_to_f_keys)
);

Academic license - for non-commercial use only


In [575]:
optimize!(m3)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 12705 rows, 28394 columns and 79367 nonzeros
Model fingerprint: 0xe294a9c9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 9767 rows and 3950 columns
Presolve time: 0.06s
Presolved: 2938 rows, 24444 columns, 70426 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 4.537e+04
 Factor NZ  : 1.592e+05 (roughly 12 MBytes of memory)
 Factor Ops : 1.193e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.81765594e+08  2.28122525e+06  1.36e+02 1.15e+03  1.21e+04     0s
   1   7.42952559e+07  7.58455046e+05  2.57e+01 1.15e+03  2.37e+03     0s
   

In [571]:
unique(value.(fire_to_fire))

2-element Vector{Float64}:
 0.0
 1.0

In [572]:
sum(value.(rest_to_fire)) + sum(value.(fire_to_fire))

235.0

In [573]:
sum(demands[!, "demand"])

223

In [453]:
r_to_r_keys

1368-element Vector{NTuple{5, Int64}}:
 (1, 43, 45, 0, 1)
 (2, 43, 45, 0, 1)
 (3, 43, 45, 0, 1)
 (4, 43, 45, 0, 1)
 (5, 43, 45, 0, 1)
 (6, 43, 45, 0, 1)
 (7, 43, 45, 0, 1)
 (8, 43, 45, 0, 1)
 (9, 43, 45, 0, 1)
 (10, 43, 45, 0, 1)
 (11, 43, 45, 0, 1)
 (12, 43, 45, 0, 1)
 (13, 43, 45, 0, 1)
 ⋮
 (19, 57, 58, 1, 1)
 (20, 57, 58, 1, 1)
 (21, 57, 58, 1, 1)
 (22, 57, 58, 1, 1)
 (23, 57, 58, 1, 1)
 (24, 57, 58, 1, 1)
 (25, 57, 58, 1, 1)
 (26, 57, 58, 1, 1)
 (27, 57, 58, 1, 1)
 (28, 57, 58, 1, 1)
 (29, 57, 58, 1, 1)
 (30, 57, 58, 1, 1)

In [445]:
rest_flow[1, 50, 1]

rest_flow[1,50,1] : -fire_to_rest[(1, 21, 49, 50, 1)] - fire_to_rest[(1, 22, 49, 50, 1)] - fire_to_rest[(1, 16, 49, 50, 1)] - fire_to_rest[(1, 20, 49, 50, 1)] - fire_to_rest[(1, 15, 49, 50, 1)] - fire_to_rest[(1, 17, 49, 50, 1)] - fire_to_rest[(1, 19, 49, 50, 1)] + rest_to_fire[(1, 19, 50, 53, 1, 1)] + rest_to_fire[(1, 20, 50, 53, 1, 1)] + rest_to_fire[(1, 21, 50, 53, 1, 1)] + rest_to_fire[(1, 15, 50, 53, 1, 1)] + rest_to_fire[(1, 17, 50, 53, 1, 1)] + rest_to_fire[(1, 22, 50, 53, 1, 1)] - rest_to_rest[(1, 48, 50, 1, 1)] + rest_to_rest[(1, 50, 52, 1, 1)] - rest_to_rest[(1, 49, 50, 1, 1)] + rest_to_rest[(1, 50, 51, 1, 1)] == 0.0

In [407]:
m3

A JuMP Model
Feasibility problem with:
Variables: 27944
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 27944 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: fire_to_fire, fire_to_rest, rest_to_fire, rest_to_rest

918-element Vector{NTuple{5, Int64}}:
 (1, 43, 45, 0, 1)
 (2, 43, 45, 0, 1)
 (3, 43, 45, 0, 1)
 (4, 43, 45, 0, 1)
 (5, 43, 45, 0, 1)
 (6, 43, 45, 0, 1)
 (7, 43, 45, 0, 1)
 (8, 43, 45, 0, 1)
 (9, 43, 45, 0, 1)
 (10, 43, 45, 0, 1)
 (11, 43, 45, 0, 1)
 (12, 43, 45, 0, 1)
 (13, 43, 45, 0, 1)
 ⋮
 (27, 55, 56, 0, 0)
 (28, 55, 56, 0, 0)
 (29, 55, 56, 0, 0)
 (30, 55, 56, 0, 0)
 (26, 56, 57, 0, 0)
 (27, 56, 57, 0, 0)
 (28, 56, 57, 0, 0)
 (29, 56, 57, 0, 0)
 (30, 56, 57, 0, 0)
 (28, 57, 58, 0, 0)
 (29, 57, 58, 0, 0)
 (30, 57, 58, 0, 0)

In [379]:
rested

13350-element Vector{NTuple{6, Int64}}:
 (1, 11, 12, 44, 45, 1)
 (2, 11, 12, 44, 45, 1)
 (3, 11, 12, 44, 45, 1)
 (4, 11, 12, 44, 45, 1)
 (5, 11, 12, 44, 45, 1)
 (6, 11, 12, 44, 45, 1)
 (7, 11, 12, 44, 45, 1)
 (8, 11, 12, 44, 45, 1)
 (9, 11, 12, 44, 45, 1)
 (10, 11, 12, 44, 45, 1)
 (11, 11, 12, 44, 45, 1)
 (12, 11, 12, 44, 45, 1)
 (13, 11, 12, 44, 45, 1)
 ⋮
 (19, 20, 25, 56, 57, 1)
 (20, 20, 25, 56, 57, 1)
 (21, 20, 25, 56, 57, 1)
 (22, 20, 25, 56, 57, 1)
 (23, 20, 25, 56, 57, 1)
 (24, 20, 25, 56, 57, 1)
 (25, 20, 25, 56, 57, 1)
 (26, 20, 25, 56, 57, 1)
 (27, 20, 25, 56, 57, 1)
 (28, 20, 25, 56, 57, 1)
 (29, 20, 25, 56, 57, 1)
 (30, 20, 25, 56, 57, 1)

In [411]:
m2 = Model(Gurobi.Optimizer)

# fire to fire travel
S0 = [(i, 
       demands[k, "fire_id"], 
       demands[l, "fire_id"], 
       demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
       demands[l, "day"],
       0
      ) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"]) &
         (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] < rest_by[i])
     ]
S1 = [(i, 
       demands[k, "fire_id"], 
       demands[l, "fire_id"], 
       demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]],
       demands[l, "day"],
       1
      ) 
      for i=1:c, k=1:n, l=1:n 
      if (demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"])
     ]

S2 = [(i, start_at[i], demands[k, "fire_id"], min_d - 1, min_d, 0) for i=1:c, k=1:n 
        if demands[k, "day"] == min_d]
S2 = [S2[x] for x in eachindex(S2)]
S = vcat(S0, S1)
S = vcat(S, S2)
@variable(m2, fire_to_fire[S] >= 0)

# fire to rest travel
S = [(i, demands[k, "fire_id"], demands[k, "day"], 
     demands[k, "day"] + tau_base_to_fire[i, demands[k, "fire_id"]], rest)
    for i=1:c, k=1:n, rest=0:1]
S = [S[x] for x in eachindex(S)]
S3 = [(i, start_at[i], min_d - 1, min_d, 0) for i=1:c]
S3 = [S3[x] for x in eachindex(S3)]
S = vcat(S, S3)
@variable(m2, fire_to_rest[S] >= 0)

# rest to fire travel
S = [(i, 
       demands[k, "fire_id"], 
       demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] - break_length,
       demands[k, "day"]
      ) 
      for i=1:c, k=1:n
      if demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]] - break_length >= min_d
     ]
@variable(m2, rest_to_fire[S] >= 0)

# rest to rest
@variable(m2, rest_to_rest[i=1:c, t=min_d-1:d+1] >= 0)

# flow constraints
f_to_f_keys_arr = [[s[1] for s in keys(fire_to_fire) if s[1][1] == i] for i=1:c]
r_to_f_keys_arr = [[s[1] for s in keys(rest_to_fire) if s[1][1] == i] for i=1:c]
f_to_r_keys_arr = [[s[1] for s in keys(fire_to_rest) if s[1][1] == i] for i=1:c]

@constraint(m2, fire_flow[i=1:c, f=min_f:f, t=min_d:d-1, rest=0:1], 
            sum(fire_to_fire[key] for key in f_to_f_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[4] == t) & (key[6] == rest)
                ) + 
    
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i]
                        if (key[1] == i) & (key[2] == f) & (key[3] == t) & (key[5] == rest)
                ) 
    ==
            sum(fire_to_fire[key] for key in f_to_f_keys_arr[i]
                        if (key[1] == i) & (key[3] == f) & (key[5] == t) & (key[6] == rest)
                ) +
    
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i]
                        if (rest == 1) & (key[1] == i) & (key[2] == f) & (key[4] == t)
                ) 
    
            )

@constraint(m2, rest_flow[i=1:c, t=min_d:d-1], 
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i]
                        if (key[1] == i) &  (key[3] == t)
                ) +
           rest_to_rest[i, t]
    
            ==             
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i]
                        if (key[1] == i) &  (key[4] == t)
                ) +
            rest_to_rest[i, t-1]
           )

f_to_f_keys = [key[1] for key in keys(fire_to_fire)]
r_to_f_keys = [key[1] for key in keys(rest_to_fire)]
@constraint(m2, dem[row=1:n], 
            sum(fire_to_fire[key] for key in f_to_f_keys
                if (key[3] == demands[row, "fire_id"]) & 
                   (key[5] == demands[row, "day"])
                ) +
            sum(rest_to_fire[key] for key in r_to_f_keys
                if (key[2] == demands[row, "fire_id"]) & 
                   (key[4] == demands[row, "day"])
                )
    >= demands[row, "demand"])

start_f_to_f_keys = [key[1] for key in keys(fire_to_fire) if key[1][4] == min_d - 1]
start_f_to_r_keys = [key[1] for key in keys(fire_to_rest) if key[1][3] == min_d - 1]
@constraint(m2, start[i=1:c],
            sum(fire_to_fire[key] for key in start_f_to_f_keys if key[1] == i) + 
            sum(fire_to_rest[key] for key in start_f_to_r_keys if key[1] == i) == 1
           )

@constraint(m2, start_rest[i=1:c], rest_to_rest[i, min_d - 1] == 0)

f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
f_to_r_keys = [s[1] for s in keys(fire_to_rest)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]
@objective(m2, Min, sum(fire_to_fire[key] * fire_dists[key[2], key[3]] for key in f_to_f_keys) +
                   sum(fire_to_rest[key] * base_fire_dists[key[1], key[2]] for key in f_to_r_keys) + 
                   sum(rest_to_fire[key] * base_fire_dists[key[1], key[2]] for key in r_to_f_keys)
)

m2

Academic license - for non-commercial use only


A JuMP Model
Minimization problem with:
Variables: 25623
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 12150 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 75 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 25623 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: dem, fire_flow, fire_to_fire, fire_to_rest, rest_flow, rest_to_fire, rest_to_rest, start, start_rest

In [412]:
optimize!(m2)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 12225 rows, 25623 columns and 70341 nonzeros
Model fingerprint: 0xa3ca3097
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 9474 rows and 3169 columns
Presolve time: 0.08s
Presolved: 2751 rows, 22454 columns, 66793 nonzeros

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 4.207e+04
 Factor NZ  : 1.437e+05 (roughly 11 MBytes of memory)
 Factor Ops : 1.041e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   8.75115755e+06  8.78272057e+03  1.71e+02 5.79e+02  3.20e+03     0s
   1   1.91228697e+06 -1.75970380e+05  3.77e+01 6.22e+02  7.26e+02     0s
   

In [416]:
rest_flow[1, 44]

rest_flow[1,44] : -fire_to_rest[(1, 26, 43, 44, 0)] + rest_to_fire[(1, 13, 44, 47)] + rest_to_fire[(1, 14, 44, 47)] + rest_to_fire[(1, 15, 44, 47)] + rest_to_fire[(1, 16, 44, 47)] + rest_to_fire[(1, 17, 44, 47)] + rest_to_fire[(1, 18, 44, 47)] + rest_to_fire[(1, 19, 44, 47)] + rest_to_fire[(1, 12, 44, 47)] - rest_to_rest[1,43] + rest_to_rest[1,44] == 0.0

In [418]:
value(rest_to_rest[1, 43])

0.0

In [413]:
value.(rest_to_rest)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(30)
    Dimension 2, 43:58
And data, a 30×16 Matrix{Float64}:
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  

In [356]:
sum(value.(rest_to_fire)) + sum(value.(fire_to_fire))

280.0

In [357]:
sum(demands[!, "demand"])

223

In [363]:
unique(value.(fire_to_fire))

2-element Vector{Float64}:
 0.0
 1.0

In [330]:
[key[1] for key in keys(fire_to_rest) if (key[1][1] == 8) & (value(fire_to_rest[key]) > 0)]

2-element Vector{NTuple{5, Int64}}:
 (8, 23, 55, 56, 1)
 (8, 34, 43, 44, 0)

In [269]:
rest_flow[16, 49]

rest_flow[16,49] : -fire_to_rest[(16, 19, 48, 49, 0)] - fire_to_rest[(16, 18, 48, 49, 0)] - fire_to_rest[(16, 20, 48, 49, 0)] - fire_to_rest[(16, 12, 48, 49, 0)] - fire_to_rest[(16, 21, 48, 49, 0)] - fire_to_rest[(16, 13, 48, 49, 0)] - fire_to_rest[(16, 19, 48, 49, 1)] - fire_to_rest[(16, 18, 48, 49, 1)] - fire_to_rest[(16, 20, 48, 49, 1)] - fire_to_rest[(16, 12, 48, 49, 1)] - fire_to_rest[(16, 21, 48, 49, 1)] - fire_to_rest[(16, 13, 48, 49, 1)] + rest_to_fire[(16, 17, 49, 52)] + rest_to_fire[(16, 15, 49, 52)] + rest_to_fire[(16, 24, 49, 52)] + rest_to_fire[(16, 19, 49, 52)] + rest_to_fire[(16, 22, 49, 52)] + rest_to_fire[(16, 21, 49, 52)] + rest_to_fire[(16, 12, 49, 52)] - rest_to_rest[16,48] + rest_to_rest[16,49] == 0.0

In [294]:
fire_flow[16, 12, 46, 1]

fire_flow[16,12,46,1] : -fire_to_fire[(16, 12, 12, 45, 46, 1)] + fire_to_fire[(16, 12, 13, 46, 47, 1)] + fire_to_fire[(16, 12, 14, 46, 47, 1)] + fire_to_fire[(16, 12, 15, 46, 47, 1)] + fire_to_fire[(16, 12, 16, 46, 47, 1)] + fire_to_fire[(16, 12, 17, 46, 47, 1)] + fire_to_fire[(16, 12, 18, 46, 47, 1)] + fire_to_fire[(16, 12, 19, 46, 47, 1)] + fire_to_fire[(16, 12, 12, 46, 47, 1)] + fire_to_rest[(16, 12, 46, 47, 1)] - rest_to_fire[(16, 12, 43, 46)] == 0.0

In [297]:
rest_flow[16, 46]

rest_flow[16,46] : -fire_to_rest[(16, 12, 45, 46, 0)] - fire_to_rest[(16, 12, 45, 46, 1)] + rest_to_fire[(16, 21, 46, 49)] + rest_to_fire[(16, 22, 46, 49)] + rest_to_fire[(16, 16, 46, 49)] + rest_to_fire[(16, 20, 46, 49)] + rest_to_fire[(16, 15, 46, 49)] + rest_to_fire[(16, 17, 46, 49)] + rest_to_fire[(16, 19, 46, 49)] - rest_to_rest[16,45] + rest_to_rest[16,46] == 0.0

In [302]:
value.(rest_to_fire[(16, 12, 43, 46)])

14.0

In [295]:
value.(fire_to_fire[(16, 12, 12, 45, 46, 1)])

0.0

In [293]:
value.(fire_to_fire[(16, 12, 12, 46, 47, 1)])

5.0

In [291]:
value.(fire_to_fire[(16, 12, 12, 47, 48, 1)])

4.0

In [279]:
value.(fire_to_rest)[(16, 12, 48, 49, 1)]

2.0

In [268]:
value.(rest_to_rest)[16, 48]

0.0

In [186]:
value(fire_to_rest[(1, 26, 43, 44, 0)])

1.0

In [305]:
value.(rest_to_rest)[16, :]

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 43:58
And data, a 16-element Vector{Float64}:
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [205]:
value(rest_to_fire[(1, 21, 48, 51)])

0.0

In [222]:
[key[1] for key in keys(fire_to_rest) if (key[1][1] == 1) & (key[1][3] == 51)]

20-element Vector{NTuple{5, Int64}}:
 (1, 19, 51, 54, 0)
 (1, 23, 51, 54, 0)
 (1, 17, 51, 54, 0)
 (1, 13, 51, 54, 0)
 (1, 22, 51, 54, 0)
 (1, 15, 51, 54, 0)
 (1, 24, 51, 54, 0)
 (1, 20, 51, 54, 0)
 (1, 12, 51, 54, 0)
 (1, 21, 51, 54, 0)
 (1, 19, 51, 54, 1)
 (1, 23, 51, 54, 1)
 (1, 17, 51, 54, 1)
 (1, 13, 51, 54, 1)
 (1, 22, 51, 54, 1)
 (1, 15, 51, 54, 1)
 (1, 24, 51, 54, 1)
 (1, 20, 51, 54, 1)
 (1, 12, 51, 54, 1)
 (1, 21, 51, 54, 1)

In [220]:
value.(fire_to_rest

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [(1, 19, 17, 51, 52, 1), (1, 23, 17, 51, 52, 1), (1, 17, 17, 51, 52, 1), (1, 13, 17, 51, 52, 1), (1, 22, 17, 51, 52, 1), (1, 15, 17, 51, 52, 1), (1, 24, 17, 51, 52, 1), (1, 20, 17, 51, 52, 1), (1, 12, 17, 51, 52, 1), (1, 21, 17, 51, 52, 1)  …  (1, 19, 12, 51, 52, 1), (1, 23, 12, 51, 52, 1), (1, 17, 12, 51, 52, 1), (1, 13, 12, 51, 52, 1), (1, 22, 12, 51, 52, 1), (1, 15, 12, 51, 52, 1), (1, 24, 12, 51, 52, 1), (1, 20, 12, 51, 52, 1), (1, 12, 12, 51, 52, 1), (1, 21, 12, 51, 52, 1)]
And data, a 70-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [196]:
[key[1] for key in keys(rest_to_fire) if key[1][1] == 1][30:50]

21-element Vector{NTuple{4, Int64}}:
 (1, 21, 47, 50)
 (1, 20, 47, 50)
 (1, 17, 47, 50)
 (1, 19, 47, 50)
 (1, 19, 48, 51)
 (1, 23, 48, 51)
 (1, 17, 48, 51)
 (1, 13, 48, 51)
 (1, 22, 48, 51)
 (1, 15, 48, 51)
 (1, 24, 48, 51)
 (1, 20, 48, 51)
 (1, 12, 48, 51)
 (1, 21, 48, 51)
 (1, 17, 49, 52)
 (1, 15, 49, 52)
 (1, 24, 49, 52)
 (1, 19, 49, 52)
 (1, 22, 49, 52)
 (1, 21, 49, 52)
 (1, 12, 49, 52)

In [309]:
maximum(value.(rest_to_rest))

1.0

In [90]:
rest_flow[29,44]

rest_flow[29,44] : -fire_to_rest[(29, 11, 44, 45)] - fire_to_rest[(29, 12, 44, 45)] + rest_to_fire[(29, 11, 43, 44)] + rest_to_fire[(29, 12, 43, 44)] == 0.0

In [53]:
S1
a

13350-element Vector{NTuple{6, Int64}}:
 (1, 11, 12, 44, 45, 1)
 (2, 11, 12, 44, 45, 1)
 (3, 11, 12, 44, 45, 1)
 (4, 11, 12, 44, 45, 1)
 (5, 11, 12, 44, 45, 1)
 (6, 11, 12, 44, 45, 1)
 (7, 11, 12, 44, 45, 1)
 (8, 11, 12, 44, 45, 1)
 (9, 11, 12, 44, 45, 1)
 (10, 11, 12, 44, 45, 1)
 (11, 11, 12, 44, 45, 1)
 (12, 11, 12, 44, 45, 1)
 (13, 11, 12, 44, 45, 1)
 ⋮
 (19, 20, 25, 56, 57, 1)
 (20, 20, 25, 56, 57, 1)
 (21, 20, 25, 56, 57, 1)
 (22, 20, 25, 56, 57, 1)
 (23, 20, 25, 56, 57, 1)
 (24, 20, 25, 56, 57, 1)
 (25, 20, 25, 56, 57, 1)
 (26, 20, 25, 56, 57, 1)
 (27, 20, 25, 56, 57, 1)
 (28, 20, 25, 56, 57, 1)
 (29, 20, 25, 56, 57, 1)
 (30, 20, 25, 56, 57, 1)

In [43]:
keys(fire_to_fire)

18423-element JuMP.Containers.DenseAxisArrayKeys{Tuple{Vector{NTuple{5, Int64}}}, JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}, 1}:
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((3, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((4, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((5, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((6, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((7, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((8, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((9, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((10, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((11, 11, 12, 44, 0),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{5, Int64}}}(((12, 11, 12, 44, 0),))
 JuMP.Containers.Dens

In [74]:
m = Model(Gurobi.Optimizer)

# crew i suppresses fire k during period t
S = [(i, demands[k, "fire_id"], demands[k, "day"]) for i=1:c, k=1:n]
@variable(m, supp[S], Bin)

# crew i rests in period t
@variable(m, rest[i=1:c, t=0:d+1], Bin)

# crew i completed a mandatory break in period t
@variable(m, break_ended[i=1:c, t=1:d], Bin)

# crew is eligible to work in period t
@variable(m, work_eligible[i=1:c, t=1:d], Bin)

# crew i rotates in period t
@variable(m, rot[i=1:c, t=1:d], Bin)

# crew i rotation eligible in period t
# (BINARY NOT NEEDED I THINK, check this)
@variable(m, rot_elig[i=1:c, t=1:d], Bin)

# fire to fire travel
S = [(i, 
       demands[k, "fire_id"], 
       demands[l, "fire_id"], 
       demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]]
      ) 
      for i=1:c, k=1:n, l=1:n 
      if demands[l, "day"] - tau[demands[k, "fire_id"], demands[l, "fire_id"]] == demands[k, "day"]
     ]
@variable(m, fire_to_fire[S] >= 0)

# fire to rest travel
S = [(i, demands[k, "fire_id"], demands[k, "day"]) for i=1:c, k=1:n]
@variable(m, fire_to_rest[S] >= 0)

# rest to fire travel
S = [(i, 
       demands[k, "fire_id"], 
       demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]]
      ) 
      for i=1:c, k=1:n
     ]
@variable(m, rest_to_fire[S] >= 0)

# rest to rest
@variable(m, rest_to_rest[i=1:c, t=0:d+1] >= 0)


# meet demand
@constraint(m, dem[row=1:n], sum(supp[s[1]] for s in keys(supp)
                                            if (s[1][2] == demands[row, "fire_id"]) & 
                                               (s[1][3] == demands[row, "day"])
                                 ) >= demands[row, "demand"])

# can only suppress if traveled there to arrive at time t
supp_keys = [s[1] for s in keys(supp)]
f_to_f_keys_arr = [[s[1] for s in keys(fire_to_fire) if s[1][1] == i] for i=1:c]
r_to_f_keys_arr = [[s[1] for s in keys(rest_to_fire) if s[1][1] == i] for i=1:c]
f_to_r_keys_arr = [[s[1] for s in keys(fire_to_rest) if s[1][1] == i] for i=1:c]
@constraint(m, depart[i in supp_keys], sum(fire_to_fire[key] for key in f_to_f_keys_arr[i[1]]
                                             if (key[2] == i[2]) & (key[4] == i[3])
                                          ) +
                                       sum(fire_to_rest[key] for key in f_to_r_keys_arr[i[1]]
                                                if (key[3] == i[3])
                                          )
                                       <= supp[i]
           )

@constraint(m, arrive[i in supp_keys], sum(fire_to_fire[key] for key in f_to_f_keys_arr[i[1]]
                                             if (key[3] == i[2]) & 
                                                (key[4] + tau[key[2], key[3]] == i[3])
                                          ) +
                                       sum(rest_to_fire[key] for key in r_to_f_keys_arr[i[1]]
                                             if (key[3] + tau_base_to_fire[key[1], key[2]] == i[3])
                                          )
                                       >= supp[i]
           )

@constraint(m, depart_rest[i=1:c, t=0:d], 
            rest_to_rest[i, t] +
            sum(rest_to_fire[key] for key in r_to_f_keys_arr[i] if key[3] == t)
            <= rest[i, t]
           )

@constraint(m, arrive_rest[i=1:c, t=1:d+1], 
            rest_to_rest[i, t-1] +
            sum(fire_to_rest[key] for key in f_to_r_keys_arr[i] 
                                 if key[3] + tau_base_to_fire[i, key[2]] == t)
            >= rest[i, t]
           );

Academic license - for non-commercial use only


In [99]:
arrive[(19, 1, 1)]

arrive[(19, 1, 1)] : -supp[(19, 1, 1)] + rest_to_fire[(19, 1, 0)] >= 0.0

In [75]:
@constraint(m, rest_start[i=1:c], rest[i, 0] == 1)

30-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 rest_start[1] : rest[1,0] == 1.0
 rest_start[2] : rest[2,0] == 1.0
 rest_start[3] : rest[3,0] == 1.0
 rest_start[4] : rest[4,0] == 1.0
 rest_start[5] : rest[5,0] == 1.0
 rest_start[6] : rest[6,0] == 1.0
 rest_start[7] : rest[7,0] == 1.0
 rest_start[8] : rest[8,0] == 1.0
 rest_start[9] : rest[9,0] == 1.0
 rest_start[10] : rest[10,0] == 1.0
 rest_start[11] : rest[11,0] == 1.0
 rest_start[12] : rest[12,0] == 1.0
 rest_start[13] : rest[13,0] == 1.0
 ⋮
 rest_start[19] : rest[19,0] == 1.0
 rest_start[20] : rest[20,0] == 1.0
 rest_start[21] : rest[21,0] == 1.0
 rest_start[22] : rest[22,0] == 1.0
 rest_start[23] : rest[23,0] == 1.0
 rest_start[24] : rest[24,0] == 1.0
 rest_start[25] : rest[25,0] == 1.0
 rest_start[26] : rest[26,0] == 1.0
 rest_start[27] : rest[27,0] == 1.0
 rest_start[28] : rest[28,0] == 1.0
 rest_start[29] 

In [76]:
m

A JuMP Model
Feasibility problem with:
Variables: 34500
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 30 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 4900 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4800 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 22590 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 11910 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: arrive, arrive_rest, break_ended, dem, depart, depart_rest, fire_to_fire, fire_to_rest, rest, rest_start, rest_to_fire, rest_to_rest, rot, rot_elig, supp, work_eligible

In [77]:
f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
f_to_r_keys = [s[1] for s in keys(fire_to_rest)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]

3000-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 1, 0)
 (2, 1, 0)
 (3, 1, 0)
 (4, 1, 0)
 (5, 1, 0)
 (6, 1, 0)
 (7, 1, 0)
 (8, 1, 0)
 (9, 1, 0)
 (10, 1, 0)
 (11, 1, 0)
 (12, 1, 0)
 (13, 1, 0)
 ⋮
 (19, 22, 58)
 (20, 22, 58)
 (21, 22, 58)
 (22, 22, 58)
 (23, 22, 58)
 (24, 22, 58)
 (25, 22, 58)
 (26, 22, 58)
 (27, 22, 58)
 (28, 22, 58)
 (29, 22, 58)
 (30, 22, 58)

In [88]:
f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
f_to_r_keys = [s[1] for s in keys(fire_to_rest)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]
@objective(m, Min, sum(fire_to_fire[key] * fire_dists[key[2], key[3]] for key in f_to_f_keys) +
                   sum(fire_to_rest[key] * base_fire_dists[key[1], key[2]] for key in f_to_r_keys) + 
                   sum(rest_to_fire[key] * base_fire_dists[key[1], key[2]] for key in r_to_f_keys)
);

275.06851918064973 fire_to_fire[(1, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(2, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(3, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(4, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(5, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(6, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(7, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(8, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(9, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(10, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(11, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(12, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(13, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(14, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(15, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(16, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(17, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(18, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(19, 2, 3, 19)] + 275.06851918064973 fire_to_fire[(20, 2, 3, 19)] + 275.06851

In [89]:
set_optimizer_attribute(m, "TimeLimit", 60)
optimize!(m)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 9730 rows, 34500 columns and 85230 nonzeros
Model fingerprint: 0x69c54ee0
Variable types: 22590 continuous, 11910 integer (11910 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]

MIP start from previous solve produced solution with objective 21866.2 (0.03s)
Loaded MIP start from previous solve with objective 21866.2

Presolve removed 2310 rows and 13590 columns
Presolve time: 0.08s
Presolved: 7420 rows, 20910 columns, 48720 nonzeros
Variable types: 16860 continuous, 4050 integer (4050 binary)

Root relaxation: objective 4.085670e+03, 5675 iterations, 0.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4085.66953    0  397 21866.1537 4085.66953  81.3%     -    0s
     0 

In [95]:
sum(value.(supp[[key for key in supp_keys if key[2] == 1]]))

1.0

In [53]:
value.(supp)[(2, 1, 1)]

1.0

In [26]:
depart[supp_keys[100]]

depart[(10, 3, 21)] : -supp[(10, 3, 21)] + fire_to_fire[(10, 3, 3, 21)] + fire_to_rest[(10, 3, 21)] <= 0.0

In [66]:
m

A JuMP Model
Feasibility problem with:
Variables: 134700
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 423 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 84630 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 50070 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: break_ended, dem, fire_to_fire, fire_to_rest, rest, rest_to_fire, rest_to_rest, rot, rot_elig, supp, work_eligible

In [59]:
r_to_r_keys_arr = [[i, [0:d]] for i=1:c]


LoadError: BoundsError: attempt to access 2-element Vector{Any} at index [4]

In [64]:
rest_to_rest[1, 1]

rest_to_rest[1,1]

30×250 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 arrive_rest[1,1] : -rest[1,1] + rest_to_rest[1,0] >= 0.0     …  arrive_rest[1,250] : -rest[1,250] + fire_to_rest[(1, 105, 249)] + rest_to_rest[1,249] >= 0.0
 arrive_rest[2,1] : -rest[2,1] + rest_to_rest[2,0] >= 0.0        arrive_rest[2,250] : -rest[2,250] + fire_to_rest[(2, 105, 249)] + rest_to_rest[2,249] >= 0.0
 arrive_rest[3,1] : -rest[3,1] + rest_to_rest[3,0] >= 0.0        arrive_rest[3,250] : -rest[3,250] + fire_to_rest[(3, 105, 249)] + rest_to_rest[3,249] >= 0.0
 arrive_rest[4,1] : -rest[4,1] + rest_to_rest[4,0] >= 0.0        arrive_rest[4,250] : -rest[4,250] + fire_to_rest[(4, 105, 249)] + rest_to_rest[4,249] >= 0.0
 arrive_rest[5,1] : -rest[5,1] + rest_to_rest[5,0] >= 0.0        arrive_rest[5,250] : -rest[5,250] + fire_to_rest[(5, 105, 249)] + rest_to_rest[5,249] >= 0.0
 arrive_rest[6,1] : -rest[6,1] + rest_t

In [80]:
arrive_rest[1, 20]

arrive_rest[1,20] : -rest[1,20] + fire_to_rest[(1, 2, 19)] + rest_to_rest[1,19] >= 0.0

In [73]:
unregister(m, :arrive)
unregister(m, :depart)
unregister(m, :depart_rest)

In [37]:
arrive[(1, 11, 42)]

arrive[(1, 11, 42)] : -supp[(1, 11, 42)] + fire_to_fire[(1, 8, 11, 41)] + fire_to_fire[(1, 9, 11, 41)] + fire_to_fire[(1, 10, 11, 41)] + rest_to_fire[(1, 11, 41)] >= 0.0

In [34]:
demands

,day,fire_id,demand,outside_gacc
,Int64,Int64,Int64,Bool
1,1,1,1,0
2,19,2,1,0
3,20,3,2,0
4,21,3,2,0
5,22,3,2,0
6,28,4,4,0
7,29,4,4,0
8,35,5,1,0
9,36,5,1,0


In [31]:
arrive[(10, 1, 1)]

arrive[(10, 1, 1)] : -supp[(10, 1, 1)] + rest_to_fire[(10, 1, 0)] >= 0.0

In [7]:
supp_keys = [s[1] for s in keys(supp)]
f_to_f_keys = [s[1] for s in keys(fire_to_fire)]
r_to_f_keys = [s[1] for s in keys(rest_to_fire)]

12690-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 1, 0)
 (2, 1, 0)
 (3, 1, 0)
 (4, 1, 0)
 (5, 1, 0)
 (6, 1, 0)
 (7, 1, 0)
 (8, 1, 0)
 (9, 1, 0)
 (10, 1, 0)
 (11, 1, 0)
 (12, 1, 0)
 (13, 1, 0)
 ⋮
 (19, 105, 248)
 (20, 105, 248)
 (21, 105, 248)
 (22, 105, 248)
 (23, 105, 248)
 (24, 105, 248)
 (25, 105, 248)
 (26, 105, 248)
 (27, 105, 248)
 (28, 105, 248)
 (29, 105, 248)
 (30, 105, 248)

In [7]:
f_to_f_keys

51720-element Vector{NTuple{4, Int64}}:
 (1, 2, 3, 19)
 (2, 2, 3, 19)
 (3, 2, 3, 19)
 (4, 2, 3, 19)
 (5, 2, 3, 19)
 (6, 2, 3, 19)
 (7, 2, 3, 19)
 (8, 2, 3, 19)
 (9, 2, 3, 19)
 (10, 2, 3, 19)
 (11, 2, 3, 19)
 (12, 2, 3, 19)
 (13, 2, 3, 19)
 ⋮
 (19, 104, 104, 187)
 (20, 104, 104, 187)
 (21, 104, 104, 187)
 (22, 104, 104, 187)
 (23, 104, 104, 187)
 (24, 104, 104, 187)
 (25, 104, 104, 187)
 (26, 104, 104, 187)
 (27, 104, 104, 187)
 (28, 104, 104, 187)
 (29, 104, 104, 187)
 (30, 104, 104, 187)

In [46]:
supp_keys = [s[1] for s in keys(supp)]
@constraint(m, arrive[i in supp_keys], sum(fire_to_fire[key[1]] for key in keys(fire_to_fire)
                                             if (key[1][3] == i[2]) & 
                                                (key[1][4] + tau[key[1][2], key[1][3]] == i[3])
                                          ) +
                                       sum(rest_to_fire[key[1]] for key in keys(rest_to_fire)
                                        if key[1][2] + tau_base_to_fire[key[1][1], key[1][2]] == i[3])
                                       >= supp[i]
           )

LoadError: An object of name arrive is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :arrive)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:arrive]`.


In [44]:
S

12690-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 1, 1)
 (2, 1, 1)
 (3, 1, 1)
 (4, 1, 1)
 (5, 1, 1)
 (6, 1, 1)
 (7, 1, 1)
 (8, 1, 1)
 (9, 1, 1)
 (10, 1, 1)
 (11, 1, 1)
 (12, 1, 1)
 (13, 1, 1)
 ⋮
 (19, 105, 249)
 (20, 105, 249)
 (21, 105, 249)
 (22, 105, 249)
 (23, 105, 249)
 (24, 105, 249)
 (25, 105, 249)
 (26, 105, 249)
 (27, 105, 249)
 (28, 105, 249)
 (29, 105, 249)
 (30, 105, 249)

In [39]:
keys(fire_to_fire)

51720-element JuMP.Containers.DenseAxisArrayKeys{Tuple{Vector{NTuple{4, Int64}}}, JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}, 1}:
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((1, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((2, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((3, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((4, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((5, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((6, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((7, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((8, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((9, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((10, 2, 3, 19),))
 JuMP.Containers.DenseAxisArrayKey{Tuple{NTuple{4, Int64}}}(((11, 2, 3, 1

In [35]:
(keys(supp)[31][1][2] == demands[2, "fire_id"]) & (keys(supp)[31][1][3] == demands[2, "day"])

true

In [32]:
demands

,day,fire_id,demand,outside_gacc
,Int64,Int64,Int64,Bool
1,1,1,1,0
2,19,2,1,0
3,20,3,2,0
4,21,3,2,0
5,22,3,2,0
6,28,4,4,0
7,29,4,4,0
8,35,5,1,0
9,36,5,1,0


In [139]:
dem = Array{JuMP.ConstraintRef}(n)

LoadError: MethodError: no method matching (Array{ConstraintRef})(::Int64)
[0mClosest candidates are:
[0m  (Array{T})([91m::DataFrameRow[39m) where T at C:\Users\mit\.julia\packages\DataFrames\BM4OQ\src\dataframerow\dataframerow.jl:414
[0m  (Array{T})([91m::AbstractDataFrame[39m) where T at C:\Users\mit\.julia\packages\DataFrames\BM4OQ\src\abstractdataframe\abstractdataframe.jl:1271
[0m  (Array{T})([91m::StaticArrays.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}[39m) where {T, S} at C:\Users\mit\.julia\packages\StaticArrays\KyWYI\src\SizedArray.jl:95
[0m  ...

In [137]:
row = 1


30-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 1, 1)
 (2, 1, 1)
 (3, 1, 1)
 (4, 1, 1)
 (5, 1, 1)
 (6, 1, 1)
 (7, 1, 1)
 (8, 1, 1)
 (9, 1, 1)
 (10, 1, 1)
 (11, 1, 1)
 (12, 1, 1)
 (13, 1, 1)
 ⋮
 (19, 1, 1)
 (20, 1, 1)
 (21, 1, 1)
 (22, 1, 1)
 (23, 1, 1)
 (24, 1, 1)
 (25, 1, 1)
 (26, 1, 1)
 (27, 1, 1)
 (28, 1, 1)
 (29, 1, 1)
 (30, 1, 1)

In [129]:
size((i, k, t) for (i, k, t) in S if k == demands[10, "fire_id"])

LoadError: MethodError: no method matching size(::Base.Iterators.Filter{var"#624#626", Matrix{Tuple{Int64, Int64, Int64}}})
[0mClosest candidates are:
[0m  size([91m::Union{QR, LinearAlgebra.QRCompactWY, QRPivoted}[39m) at C:\Users\mit\AppData\Local\Programs\Julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:567
[0m  size([91m::Union{QR, LinearAlgebra.QRCompactWY, QRPivoted}[39m, [91m::Integer[39m) at C:\Users\mit\AppData\Local\Programs\Julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\qr.jl:566
[0m  size([91m::Union{Cholesky, CholeskyPivoted}[39m) at C:\Users\mit\AppData\Local\Programs\Julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\cholesky.jl:494
[0m  ...

In [105]:
S = [(demands[k, "fire_id"], demands[k, "day"]) for k=1:n]

423-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 19)
 (3, 20)
 (3, 21)
 (3, 22)
 (4, 28)
 (4, 29)
 (5, 35)
 (5, 36)
 (6, 38)
 (7, 39)
 (8, 41)
 (9, 41)
 ⋮
 (102, 183)
 (101, 183)
 (102, 184)
 (101, 184)
 (102, 185)
 (101, 185)
 (101, 186)
 (102, 186)
 (103, 186)
 (104, 187)
 (104, 188)
 (105, 249)

In [101]:
dems = [demands[k, "demand"] for k=1:n]

423-element Vector{Int64}:
 1
 1
 2
 2
 2
 4
 4
 1
 1
 1
 2
 1
 2
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 2
 1
 1
 1

In [90]:
S = [(i, 
       demands[k, "fire_id"], 
       demands[k, "day"] - tau_base_to_fire[i, demands[k, "fire_id"]]
      ) 
      for i=1:c, k=1:n
     ]

30×423 Matrix{Tuple{Int64, Int64, Int64}}:
 (1, 1, 0)   (1, 2, 18)   (1, 3, 19)   …  (1, 104, 187)   (1, 105, 248)
 (2, 1, 0)   (2, 2, 18)   (2, 3, 19)      (2, 104, 187)   (2, 105, 248)
 (3, 1, 0)   (3, 2, 18)   (3, 3, 19)      (3, 104, 187)   (3, 105, 248)
 (4, 1, 0)   (4, 2, 18)   (4, 3, 19)      (4, 104, 187)   (4, 105, 248)
 (5, 1, 0)   (5, 2, 18)   (5, 3, 19)      (5, 104, 187)   (5, 105, 248)
 (6, 1, 0)   (6, 2, 18)   (6, 3, 19)   …  (6, 104, 187)   (6, 105, 248)
 (7, 1, 0)   (7, 2, 18)   (7, 3, 19)      (7, 104, 187)   (7, 105, 248)
 (8, 1, 0)   (8, 2, 18)   (8, 3, 19)      (8, 104, 187)   (8, 105, 248)
 (9, 1, 0)   (9, 2, 18)   (9, 3, 19)      (9, 104, 187)   (9, 105, 248)
 (10, 1, 0)  (10, 2, 18)  (10, 3, 19)     (10, 104, 187)  (10, 105, 248)
 (11, 1, 0)  (11, 2, 18)  (11, 3, 19)  …  (11, 104, 187)  (11, 105, 248)
 (12, 1, 0)  (12, 2, 18)  (12, 3, 19)     (12, 104, 187)  (12, 105, 248)
 (13, 1, 0)  (13, 2, 18)  (13, 3, 19)     (13, 104, 187)  (13, 105, 248)
 ⋮               

In [92]:
demands

,day,fire_id,demand,outside_gacc
,Int64,Int64,Int64,Bool
1,1,1,1,0
2,19,2,1,0
3,20,3,2,0
4,21,3,2,0
5,22,3,2,0
6,28,4,4,0
7,29,4,4,0
8,35,5,1,0
9,36,5,1,0


In [82]:
S = [(i, demands[k, "fire_id"], demands[k, "day"]) for i=1:c, k=1:n]

30×423 Matrix{Tuple{Int64, Int64, Int64}}:
 (1, 1, 1)   (1, 2, 19)   (1, 3, 20)   …  (1, 104, 188)   (1, 105, 249)
 (2, 1, 1)   (2, 2, 19)   (2, 3, 20)      (2, 104, 188)   (2, 105, 249)
 (3, 1, 1)   (3, 2, 19)   (3, 3, 20)      (3, 104, 188)   (3, 105, 249)
 (4, 1, 1)   (4, 2, 19)   (4, 3, 20)      (4, 104, 188)   (4, 105, 249)
 (5, 1, 1)   (5, 2, 19)   (5, 3, 20)      (5, 104, 188)   (5, 105, 249)
 (6, 1, 1)   (6, 2, 19)   (6, 3, 20)   …  (6, 104, 188)   (6, 105, 249)
 (7, 1, 1)   (7, 2, 19)   (7, 3, 20)      (7, 104, 188)   (7, 105, 249)
 (8, 1, 1)   (8, 2, 19)   (8, 3, 20)      (8, 104, 188)   (8, 105, 249)
 (9, 1, 1)   (9, 2, 19)   (9, 3, 20)      (9, 104, 188)   (9, 105, 249)
 (10, 1, 1)  (10, 2, 19)  (10, 3, 20)     (10, 104, 188)  (10, 105, 249)
 (11, 1, 1)  (11, 2, 19)  (11, 3, 20)  …  (11, 104, 188)  (11, 105, 249)
 (12, 1, 1)  (12, 2, 19)  (12, 3, 20)     (12, 104, 188)  (12, 105, 249)
 (13, 1, 1)  (13, 2, 19)  (13, 3, 20)     (13, 104, 188)  (13, 105, 249)
 ⋮               

In [50]:
supp

JuMP.Containers.SparseAxisArray{VariableRef, 3, Tuple{Int64, Int64, Int64}} with 12690 entries:
  [12, 25, 70 ]  =  supp[12,25,70]
  [13, 26, 60 ]  =  supp[13,26,60]
  [14, 23, 50 ]  =  supp[14,23,50]
  [14, 77, 132]  =  supp[14,77,132]
  [15, 30, 70 ]  =  supp[15,30,70]
  [19, 50, 110]  =  supp[19,50,110]
  [2, 78, 128 ]  =  supp[2,78,128]
  [21, 80, 130]  =  supp[21,80,130]
  [22, 68, 131]  =  supp[22,68,131]
  [23, 48, 111]  =  supp[23,48,111]
  [23, 49, 109]  =  supp[23,49,109]
                 ⋮
  [24, 31, 82 ]  =  supp[24,31,82]
  [24, 68, 123]  =  supp[24,68,123]
  [25, 31, 84 ]  =  supp[25,31,84]
  [26, 24, 51 ]  =  supp[26,24,51]
  [28, 21, 61 ]  =  supp[28,21,61]
  [3, 27, 67  ]  =  supp[3,27,67]
  [5, 60, 127 ]  =  supp[5,60,127]
  [6, 67, 125 ]  =  supp[6,67,125]
  [7, 21, 68  ]  =  supp[7,21,68]
  [7, 50, 110 ]  =  supp[7,50,110]

In [16]:
demands

,day,fire_id,demand,outside_gacc
,Int64,Int64,Int64,Bool
1,1,1,1,0
2,19,2,1,0
3,20,3,2,0
4,21,3,2,0
5,22,3,2,0
6,28,4,4,0
7,29,4,4,0
8,35,5,1,0
9,36,5,1,0
